# Explore
Sometimes I just want to understand better what a layer does. In this notebook I have the structure for exploring various layers.

## Input shape

All layers derive from the base `Layer` class whose constructor accepts the `input_shape` argument, so all layers will accept this argument even if not explicitly mentioned in the docs. Input shape is the shape of a single sample in the input. The actual input will be an array of such samples. Unlike TF, I don't have to say (None, rows, cols). The None is implied. But when I send input to this layer, it will accept tensors of shape (None, rows, cols).

## Flatten
Takes each element of the input tensor row-by-row and lays it side by side.

In [9]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Flatten

l0 = Input(shape=(2, 2, 3), dtype='int32')
l1 = Flatten()(l0)
model = Model(l0, l1)
# No need to compile the model as we are not going to train it
model.summary()
x1 = np.array([[[1, 2, 3], [4, 5, 6]],
               [[7, 8, 9], [10, 11, 12]]])
x2 = np.array([[[13, 14, 15], [16, 17, 18]],
               [[19, 20, 21], [22, 23, 24]]])
X = np.array([x1, x2])
model.predict(X)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 2, 2, 3)           0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 12)                0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
None
[[ 1  2  3  4  5  6  7  8  9 10 11 12]
 [13 14 15 16 17 18 19 20 21 22 23 24]]


## Concatenate
It takes as input the output tensors of two layers, and lays them side by side. The preceeding layers are usually `Flatten`, so it makes sense to explore what `concatenate` does for simple row vectors. If layer 1 outputs [1, 1, 1] and layer 2 outputs [10, 10, 10, 10], the concatenation layer will output [1, 1, 1, 10, 10, 10, 10]. The two input tensors do not have to be of the same size.

In [10]:
import numpy as np
from keras import layers
from keras import Input
from keras.models import Model

a = Input(shape=(3,), dtype='int32', name='a')
b = Input(shape=(5,), dtype='int32', name='b')
c = layers.concatenate([a, b], axis=-1)
model = Model([a, b], c)
model.summary()
xa = np.array([[1, 1, 1],
               [2, 2, 2]])
xb = np.array([[10, 10, 10, 10, 10],
               [20, 20, 20, 20, 20]])
model.predict([xa, xb])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
a (InputLayer)                   (None, 3)             0                                            
____________________________________________________________________________________________________
b (InputLayer)                   (None, 5)             0                                            
____________________________________________________________________________________________________
concatenate_1 (Concatenate)      (None, 8)             0           a[0][0]                          
                                                                   b[0][0]                          
Total params: 0
Trainable params: 0
Non-trainable params: 0
____________________________________________________________________________________________________


array([[ 1,  1,  1, 10, 10, 10, 10, 10],
       [ 2,  2,  2, 20, 20, 20, 20, 20]], dtype=int32)

## Add
It takes in as input the output of a number of layers and does an element-wise addition (essentially tensor addition). Usually the preceeding layers are `Flatten` so it makes sense to explore this for simple row vectors. Unlike `concatenate`, the `add` operation needs tensors of the same size.

In [11]:
import numpy as np
from keras import layers, Input
from keras.models import Model

a_in = Input(shape=(3,), dtype='int32', name='a')
b_in = Input(shape=(3,), dtype='int32', name='b')
c_in = Input(shape=(3,), dtype='int32', name='c')
d_out = layers.add([a_in, b_in, c_in])
model = Model([a_in, b_in, c_in], d_out)
model.summary()
a = np.array([[1, 1, 1],
               [2, 2, 2]])
b = np.array([[10, 10, 10],
               [20, 20, 20]])
c = np.array([[100, 100, 100],
              [200, 200, 200]])
model.predict([a, b, c])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
a (InputLayer)                   (None, 3)             0                                            
____________________________________________________________________________________________________
b (InputLayer)                   (None, 3)             0                                            
____________________________________________________________________________________________________
c (InputLayer)                   (None, 3)             0                                            
____________________________________________________________________________________________________
add_1 (Add)                      (None, 3)             0           a[0][0]                          
                                                                   b[0][0]                 

array([[111, 111, 111],
       [222, 222, 222]], dtype=int32)

## RepeatVector
Takes in a tensor and copies it the specified number of times.

In [14]:
from keras.models import Model
from keras.layers import RepeatVector, Input

l0 = Input(shape=(3,), dtype='int32')
l1 = RepeatVector(2)(l0)
model = Model(l0, l1)
model.summary()
X = np.array([[1, 1, 1], 
              [2, 2, 2], 
              [3, 3, 3]])
y = model.predict(X)
print(y.shape)
print(y)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 3)                 0         
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 2, 3)              0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
(3, 2, 3)
[[[1 1 1]
  [1 1 1]]

 [[2 2 2]
  [2 2 2]]

 [[3 3 3]
  [3 3 3]]]


## Convolution

In [15]:
from keras.models import Model
from keras.layers import Input, Conv2D, Conv2DTranspose
from keras import backend as K
import numpy as np

In [25]:
def my_init(shape):
    w = np.arange(1, 10, dtype=np.float).reshape(shape)
    return K.variable(w)

x = Input(shape=(4, 4, 1), dtype='float')
y = Conv2D(1, 3, kernel_initializer=my_init)(x)
model = Model(x, y)
model.summary()

x_val = np.arange(1, 17, dtype=np.float).reshape(1, 4, 4, 1)
y_val = model.predict(x_val)
print(y_val)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 4, 4, 1)           0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 2, 2, 1)           10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
[[[[348.]
   [393.]]

  [[528.]
   [573.]]]]


In [27]:
yt = Input(shape=(2, 2, 1), dtype='float')
xt = Conv2DTranspose(1, 3, kernel_initializer=my_init)(yt)
modelt = Model(yt, xt)
modelt.summary()

x_val = modelt.predict(y_val)
print(x_val)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        (None, 2, 2, 1)           0         
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 4, 4, 1)           10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
[[[[  348.]
   [ 1089.]
   [ 1830.]
   [ 1179.]]

  [[ 1920.]
   [ 4941.]
   [ 6783.]
   [ 4077.]]

  [[ 4548.]
   [10467.]
   [12309.]
   [ 6975.]]

  [[ 3696.]
   [ 8235.]
   [ 9336.]
   [ 5157.]]]]


## Layers

In [53]:
from keras.models import Model
from keras.layers import Input, Dense
from keras import backend as K
import keras
import numpy as np
import tensorflow as tf

In [43]:
def my_init(shape):
    w = np.eye(shape[0])
    return K.variable(w)

x = Input(shape=(3,), dtype='float')
y = Dense(3, use_bias=False, kernel_initializer=my_init)(x)
model = Model(x, y)
model.summary()

x_val = np.array([1, 2, 3], dtype=np.float).reshape(1, 3)
y_val = model.predict(x_val)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_32 (InputLayer)        (None, 3)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 9         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________
[[1. 2. 3.]]


In [63]:
y_true = np.array([1., 0., 1.])
y_pred = np.array([0.9, 0.7, 0.8])

In [64]:
y_true_t = K.variable(y_true)
y_pred_t = K.variable(y_pred)
tp = keras.losses.binary_crossentropy(y_true_t, y_pred_t)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    loss = tp.eval()
print(loss)

0.51082563


In [77]:
- np.mean((y_true*np.log(y_pred) + (1-y_true)*np.log(1-y_pred)))

0.5108256237659906